In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

In [2]:
df = pd.read_csv("data/metrics.csv")
# df = pd.read_csv("data/antlr4.csv")
# df = pd.read_csv("data/discord4j.csv")
# df = pd.read_csv("data/junit4.csv")
df.head()

,Project,ID,Name,LongName,Parent,Component,Path,Line,Column,EndLine,...,Documentation Metric Rules,Documentation Rules,Error Prone Rules,Inheritance Metric Rules,Multithreading Rules,Performance Rules,Security Rules,Size Metric Rules,isDefective,Unnamed: 92
0,antlr4,L134,Tool,org.antlr.v4.Tool,L500,L103,C:\Users\domco\Desktop\DP\repos\antlr4\tool\sr...,89,1,859,...,80,0,4,0,0,18,0,21,1,NaN
1,antlr4,L125,Antlr4ErrorLog,org.antlr.mojo.antlr4.Antlr4ErrorLog,L107,L103,C:\Users\domco\Desktop\DP\repos\antlr4\antlr4-...,47,1,124,...,0,0,0,0,0,0,0,0,0,NaN
2,antlr4,L199,Antlr4Mojo,org.antlr.mojo.antlr4.Antlr4Mojo,L107,L103,C:\Users\domco\Desktop\DP\repos\antlr4\antlr4-...,77,1,505,...,26,0,2,0,0,1,0,10,0,NaN
3,antlr4,L643,ANTLRFileStream,org.antlr.v4.runtime.ANTLRFileStream,L507,L103,C:\Users\domco\Desktop\DP\repos\antlr4\runtime...,42,1,86,...,11,0,1,0,0,1,0,2,0,NaN
4,antlr4,L645,ANTLRInputStream,org.antlr.v4.runtime.ANTLRInputStream,L507,L103,C:\Users\domco\Desktop\DP\repos\antlr4\runtime...,47,1,246,...,33,2,0,0,0,1,0,7,0,NaN


## Take the working set of reatures
NOA: Number of Attributes <br>
NOP: Number of Parameters <br>
NOC: Number of Children <br>
CBO: Coupling Between Objects <br>
DIT: Depth Inheritance Tree <br>
RFC: Response for a Class <br>
LCOM5: Lack of Cohesion of Methods <br>
<i>Labels</i><br>
LOC: Line of Code <br>

In [3]:
# Labels are the values we want to predict
labels = np.array(df['LOC'])

# Remove the labels from the working dataset
features = df[['NOA', 'NOP', 'NOC', 'CBO', 'DIT', 'RFC', 'LCOM5']]

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [4]:
# Split the data into training and testing set
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42) 

# Quick sanity check with the shapes of Training and testing datasets
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1679, 7)
Training Labels Shape: (1679,)
Testing Features Shape: (720, 7)
Testing Labels Shape: (720,)


## Traine model

In [5]:
# Initialize AdaBoost regressor with a decision stump as the base estimator
base_estimator = DecisionTreeRegressor(max_depth=1)
n_estimators = 50
ada_regressor = AdaBoostRegressor(base_estimator=base_estimator, 
                                  n_estimators=n_estimators)

# Train the regressor
AB = ada_regressor.fit(train_features, train_labels)

## Make prediction

In [6]:
#Measuring Goodness of fit in Training data
print('R2 Value:', metrics.r2_score(train_labels, AB.predict(train_features)))

# Make predictions
predictions = ada_regressor.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'LOC')

R2 Value: 0.6257909063234387
Mean Absolute Error: 47.16 LOC


## Finding the accuracy of the model
Using the Mean Absolute Percentage Error (MAPE)

In [7]:
# Calculate mean absolute percentage error (MAPE)
MAPE = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(MAPE)
print('Accuracy:', round(accuracy, 2), '%')

Accuracy: -280.37 %


In [8]:
TestingData = pd.DataFrame(data=test_features, columns=feature_list)
TestingData['LOC'] = test_labels
TestingData['PredictedLOC'] = predictions
TestingData.head()

,NOA,NOP,NOC,CBO,DIT,RFC,LCOM5,LOC,PredictedLOC
0,0,0,0,4,0,3,2,14,46.218527
1,0,0,0,3,0,3,3,17,46.218527
2,4,1,1,3,4,4,0,36,46.218527
3,0,0,0,0,0,0,0,10,46.218527
4,0,0,0,5,0,4,2,25,46.218527
